In [165]:
import pandas
import numpy
import random
import sys

from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM, GRU
from keras.optimizers import RMSprop, Adam



In [163]:
cd ..

/Users/mmccurdy/code/starwars-namer


In [164]:
from swnamer.process import chunk_names, create_indices

In [130]:
legend_names = pandas.read_csv('../data/legend_names.csv')

In [131]:
canon_names = pandas.read_csv('../data/cannon_names.csv')

In [132]:
cast_names = pandas.read_csv('../data/cast_names.csv')

In [133]:
clone_wars_names = pandas.read_csv('../data/clone_wars.csv')

In [134]:
kotor_names = pandas.read_csv('../data/kotor.csv')

In [135]:
combined = pandas.concat((legend_names, canon_names, cast_names, clone_wars_names, kotor_names))

In [136]:
combined = combined.reset_index(drop=True)

In [137]:
combined.shape

(1101, 1)

In [138]:
combined.drop_duplicates().shape

(763, 1)

In [139]:
combined = combined.drop_duplicates()

In [140]:
combined

,name
0,8t88
1,Abeloth
2,King Adas
3,Darth Andeddu
4,Nom Anor
5,Bail Antilles
6,Arcann
7,Seti Ashgad
8,Attichitcuk
9,Tavion Axmis


In [141]:
combined.loc[:, 'name'] = combined.name.str.lower()

In [142]:
combined.loc[:, 'length'] = combined.name.str.len()

In [143]:
combined.length.max()

27

In [144]:
combined[combined.length == 27]

,name,length
166,kybo ren (gir kybo ren-cha),27
314,lieutenant kaydel ko connix,27
482,admiral conan antonio motti,27
874,queen breha antilles organa,27


In [145]:
combined.to_csv('../output/starwars_processed.csv', index=False)

In [166]:
token_to_index, index_to_token = create_indices(combined, 'name')

In [167]:
chars = token_to_index.keys()

In [168]:
vocab_size = len(chars)
vocab_size

48

In [178]:
timesteps = 1

In [179]:
chunks, next_char = chunk_names(combined, 'name', timesteps)

In [180]:
X = numpy.zeros((len(chunks), timesteps, vocab_size))
y = numpy.zeros((len(chunks), vocab_size))

In [181]:
for i, chunk in enumerate(chunks):
    for t, token in enumerate(chunk):
        index = token_to_index[token]
        X[i, t, index] = 1
    index = token_to_index[next_char[i]]
    y[i, index] = 1

In [183]:
model = Sequential()
model.add(GRU(128, input_shape=(timesteps, vocab_size), return_sequences=True))
model.add(GRU(128))
model.add(Dense(vocab_size))
model.add(Activation('softmax'))
optimizer = RMSprop(lr=.1, clipvalue=12)
model.compile(optimizer, 'categorical_crossentropy')

In [184]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = numpy.asarray(preds).astype('float64')
    preds = numpy.log(preds) / temperature
    exp_preds = numpy.exp(preds)
    preds = exp_preds / numpy.sum(exp_preds)
    probas = numpy.random.multinomial(1, preds, 1)
    return numpy.argmax(probas)


def on_epoch_end(epoch, logs):
    if epoch % 10 != 0:
        return
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(chunks) - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = chunks[start_index]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(50):
            x_pred = numpy.zeros((1, timesteps, vocab_size))
            for t, char in enumerate(sentence):
                x_pred[0, t, token_to_index[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = index_to_token[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char
            
            if next_char == '\n':
                break

            sys.stdout.write(next_char)
            sys.stdout.flush()
    
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [185]:
model.fit(X, y, epochs=200, batch_size=128, callbacks=[print_callback])

Epoch 1/200
8446/8446 [==============================] - 2s 224us/step - loss: 3.9616

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "a"
ararrrrrrarrrrrarrrrrrrrrararararrrrarrrrrrrarrarar
----- diversity: 0.5
----- Generating with seed: "a"
as
----- diversity: 1.0
----- Generating with seed: "a"
arsssanelleradin s
----- diversity: 1.2
----- Generating with seed: "a"
ararernepeanga
Epoch 2/200
8446/8446 [==============================] - 1s 61us/step - loss: 3.2440
Epoch 3/200
8446/8446 [==============================] - 1s 62us/step - loss: 3.1276
Epoch 4/200
8446/8446 [==============================] - 1s 64us/step - loss: 3.0491
Epoch 5/200
8446/8446 [==============================] - 1s 61us/step - loss: 3.0803
Epoch 6/200
8446/8446 [==============================] - 1s 59us/step - loss: 3.0543
Epoch 7/200
8446/8446 [==============================] - 1s 63us/step - loss: 3.0725
Epoch 8/200
8446/8446 [==============================] - 1s 69us/

/Users/mmccurdy/.virtualenvs/starwars-namer/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


s
Epoch 52/200
8446/8446 [==============================] - 1s 60us/step - loss: 3.7223
Epoch 53/200
8446/8446 [==============================] - 1s 60us/step - loss: 3.7065
Epoch 54/200
8446/8446 [==============================] - 0s 58us/step - loss: 3.6918
Epoch 55/200
8446/8446 [==============================] - 1s 61us/step - loss: 3.6520
Epoch 56/200
8446/8446 [==============================] - 0s 57us/step - loss: 3.6884
Epoch 57/200
8446/8446 [==============================] - 1s 61us/step - loss: 3.6552
Epoch 58/200
8446/8446 [==============================] - 1s 60us/step - loss: 3.6337
Epoch 59/200
8446/8446 [==============================] - 1s 62us/step - loss: 3.6819
Epoch 60/200
8446/8446 [==============================] - 1s 61us/step - loss: 3.6346
Epoch 61/200
8446/8446 [==============================] - 1s 60us/step - loss: 3.6522

----- Generating text after Epoch: 60
----- diversity: 0.2
----- Generating with seed: " "
 de de
----- diversity: 0.5
----- Generating w

wer ro bo baelffffr jereeraa
----- diversity: 1.0
----- Generating with seed: "w"
we blron baloole dammorraeriaamioornorin da jooun t
----- diversity: 1.2
----- Generating with seed: "w"
we
Epoch 122/200
8446/8446 [==============================] - 1s 61us/step - loss: 3.5696
Epoch 123/200
8446/8446 [==============================] - 0s 58us/step - loss: 3.5763
Epoch 124/200
8446/8446 [==============================] - 1s 70us/step - loss: 3.5755
Epoch 125/200
8446/8446 [==============================] - 0s 57us/step - loss: 3.5752
Epoch 126/200
8446/8446 [==============================] - 1s 63us/step - loss: 3.5723
Epoch 127/200
8446/8446 [==============================] - 0s 58us/step - loss: 3.5746
Epoch 128/200
8446/8446 [==============================] - 0s 58us/step - loss: 3.5853
Epoch 129/200
8446/8446 [==============================] - 0s 55us/step - loss: 3.5683
Epoch 130/200
8446/8446 [==============================] - 1s 64us/step - loss: 3.5757
Epoch 131/200
8446/8446 [==

ariliririreris
----- diversity: 0.5
----- Generating with seed: "a"
aner
----- diversity: 1.0
----- Generating with seed: "a"
air
----- diversity: 1.2
----- Generating with seed: "a"
ami beywacgoiodedd
Epoch 192/200
8446/8446 [==============================] - 0s 57us/step - loss: 3.5821
Epoch 193/200
8446/8446 [==============================] - 0s 56us/step - loss: 3.5918
Epoch 194/200
8446/8446 [==============================] - 0s 56us/step - loss: 3.5701
Epoch 195/200
8446/8446 [==============================] - 0s 57us/step - loss: 3.5716
Epoch 196/200
8446/8446 [==============================] - 0s 59us/step - loss: 3.5733
Epoch 197/200
8446/8446 [==============================] - 0s 55us/step - loss: 3.5766
Epoch 198/200
8446/8446 [==============================] - 1s 64us/step - loss: 3.5872
Epoch 199/200
8446/8446 [==============================] - 0s 58us/step - loss: 3.5737
Epoch 200/200
8446/8446 [==============================] - 0s 55us/step - loss: 3.5892
